In [1]:
import numpy as np

from utils import timestamp, YEAR
from utils.arrayview import ArrayView, TimeseriesView

from prediction.models.fit_model import TSModel
from prediction.models.model_parameters import ModelParameters
from prediction.models.factor_management import FactorList
from prediction.models.preprocessing import load_slices, print_factor_order

from prediction.models.prediction import factornames_trimmed
from prediction.models.parameters import factor_build_end

In [2]:
av = ArrayView.from_file('../datadev/brain_final2cut.av.bcolz')

In [3]:
av_new = ArrayView.from_file('../datadev/roll_normspeed.av.bcolz')

In [4]:
av.norm_speed = av_new.residuals

In [5]:
factornames_trimmed += ['norm_speed']

In [6]:
tsav = load_slices(path='../datadev/')

In [7]:
pars = ModelParameters(av, oos_start=factor_build_end+YEAR, depth=3, lmbd=10, verbose=True)

In [8]:
fl = FactorList(av, factornames_trimmed)

In [9]:
fl.preprocess(pars)
factors = fl.asmatrix()

INFO:models:Getting factors from av and rescaling...


.

 .

 .

 .

...

 .

 .

 .

 .

 .

In [10]:
tsmod = TSModel(factors, tsav, pars)

In [11]:
tsmod.fit_slices()

INFO:models:Fitting slice 0


prediction/models/fit_model.py:153: RuntimeWarning: divide by zero encountered in log
  stats, self.step2probs[sl] = self.concat_and_fit(strata, result, nonrunner, [fback, flay, np.log(probs)], ts_idx, valid2, verbose=False, step=2)


INFO:models:Fitting slice 1


INFO:models:Fitting slice 2


INFO:models:Fitting slice 3


...

INFO:models:Fitting slice 5


INFO:models:Fitting slice 6


INFO:models:Fitting slice 7


INFO:models:Fitting slice 8


INFO:models:Fitting slice 9


In [12]:
tsmod.stats1.ll

array([[-1598.28109766, -1598.28109766, -1859.74420536],
       [-1601.44944771, -1601.44944771, -1854.89375297],
       [-1605.13603507, -1605.13603507, -1845.51741867],
       [-1607.40698225, -1607.40698225, -1831.42637193],
       [-1618.37190854, -1618.37190854, -1801.41372198],
       [-1629.55094894, -1629.55094894, -1784.57238093],
       [-1646.80949982, -1646.80949982, -1758.55506875],
       [-1657.36578255, -1657.36578255, -1744.07935962],
       [-1657.92391309, -1657.92391309, -1744.25509294],
       [-1654.37918039, -1654.37918039, -1738.79958448],
       [           nan,            nan,            nan]])

In [13]:
tsmod.stats2.ll

array([[-1590.76898559, -1859.87312704, -1859.87312704],
       [-1593.95784573, -1855.04778177, -1855.04778177],
       [-1597.70998344, -1845.64807847, -1845.64807847],
       [-1599.65185616, -1831.53819057, -1831.53819057],
       [-1610.86168508, -1801.62224471, -1801.62224471],
       [-1622.19258857, -1785.11804839, -1785.11804839],
       [-1639.04469989, -1761.09487658, -1761.09487658],
       [-1650.02498855, -1748.27318986, -1748.27318986],
       [-1650.88424885, -1749.06669001, -1749.06669001],
       [-1647.84956341, -1747.06695468, -1747.06695468],
       [           nan,            nan,            nan]])

In [14]:
print_factor_order(tsmod.stats1, factornames_trimmed)

  0:                                                              norm_speed    t-score sum: 1206.93
  1:                                                              zec3f8f8b9    t-score sum: 342.66
  2:                                                              zce609d953    t-score sum: 154.46
  3:                                                              z77c9cc0a5    t-score sum: 150.17
  4:                                                              zec0c22a48    t-score sum: 129.06
  5:                                                              za5cf91a5d    t-score sum: 103.71
  6:                                                              z4fdabfabc    t-score sum: 98.65
  7:                                                              z0f50ce7c0    t-score sum: 91.97
  8:                                                              z6f1eaa94d    t-score sum: 71.98
  9:                                                              z8ca2ea7f2    t-score sum: 71.55
 10

In [15]:
tsmod.write_simdata('simdata.p')